In [1]:
%matplotlib inline

In [2]:
from kaggle_quora_question_pairs_common import *

dataset.hdf
sample_submission.csv
sample_submission.csv.zip
test.csv
test.csv.zip
train.csv
train.csv.zip



/home/avsolatorio/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
%%time
train_df, test_df = load_train_test()
unique_questions = get_unique_questions(train_df, test_df, include_test=True)

CPU times: user 5.94 s, sys: 208 ms, total: 6.15 s
Wall time: 6.15 s


In [4]:
my_sentences = unique_questions[:100000].str.replace('?', '').replace('.', '').str.replace('\W', ' ').str.split()

In [5]:
# %%time

# def train_word2vec(
#     tokenized_questions,
#     pre_trained_model='GoogleNews-vectors-negative300.bin.gz'
#     size=300,
#     iter=100,
#     min_count=1,
#     negative=10,
#     workers=7,
#     min_alpha=0.0001,
#     window=5
# ):
#     # https://github.com/RaRe-Technologies/gensim/issues/1245
#     # List of tokenized questions.
#     # e.g. ['What', 'is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india']
#     # pre_trained_model can be any pre trained model that gensim accepts, e.g., Glove or GoogleNews word2vec

#     # Initialize model
#     word_vectors = Word2Vec(
#         size=size, iter=iter, min_count=min_count, negative=negative, workers=workers,
#         min_alpha=min_alpha, window=window,
#     )

#     # Initialize vocab
#     word_vectors.build_vocab(my_sentences)

#     # Initialize vectors in local model with with vectors from pre-trained model with overlapping vocabulary.
#     # Set `lockf` to 1 for re-training
#     word_vectors.intersect_word2vec_format(pre_trained_model, lockf=1, binary=True)

#     # Adjust pre-trained vectors to adapt its distribution with that of the local data via retraining.
#     word_vectors.train(tokenized_questions)

#     return word_vectors

In [6]:
%%time
word_vectors = train_word2vec(my_sentences)

CPU times: user 12min 20s, sys: 9.57 s, total: 12min 30s
Wall time: 4min 10s


In [135]:
my_sentences[3287:3289]

3287                                    []
3288    [How, do, you, trace, your, roots]
dtype: object

In [137]:
np.array([np.array([word_vectors[i] for i in st]).mean(axis=0) if st else np.zeros(300) for st in my_sentences[3285:3290]]).shape

(5, 300)

In [150]:
%%time
cos_dist = einsum_pairwise_cos_sim(
    np.array([np.array([word_vectors[i] for i in st]).mean(axis=0) if st else np.random.randn(300) for st in my_sentences[:10000]]),
    np.array([np.array([word_vectors[i] for i in st]).mean(axis=0) if st else np.random.randn(300) for st in my_sentences[10000:20000]]),
#     binary_input=False
)

CPU times: user 628 ms, sys: 12 ms, total: 640 ms
Wall time: 640 ms


In [107]:
word_vectors.most_similar('buxar', topn=20)

KeyError: "word 'buxar' not in vocabulary"

In [35]:
word_vectors.most_similar('Kohinoor', topn=20)

[('Noor', 0.4805505871772766),
 ('Meghnath', 0.4656824469566345),
 ('Koh', 0.44670557975769043),
 ('Jaya', 0.4466068148612976),
 ('Kingsley', 0.44049006700515747),
 ('Chhotepur', 0.4382625222206116),
 ('dhampir', 0.433312326669693),
 ('Mumtaz', 0.4214532971382141),
 ('Mormont', 0.41787993907928467),
 ('rani', 0.41724181175231934),
 ('Mughal', 0.41038641333580017),
 ('Divya', 0.40834560990333557),
 ('Aadam', 0.4072263836860657),
 ('Nagarjun', 0.40593525767326355),
 ('Kunti', 0.4028519093990326),
 ('Anaximander', 0.3990257680416107),
 ('sanyogita', 0.39638152718544006),
 ('Warframe', 0.396377831697464),
 ('Biswa', 0.3936367630958557),
 ('Maharana', 0.39177757501602173)]

In [172]:
word_vectors.most_similar('step', topn=20)

[('Sigmoid', 0.3332473337650299),
 ('oberving', 0.32975703477859497),
 ('ramp', 0.3224278390407562),
 ('prevoius', 0.314231276512146),
 ('HDDs', 0.2744296193122864),
 ('cassandra', 0.273955762386322),
 ('encouraging', 0.2650046944618225),
 ('oneself', 0.2644054889678955),
 ('standardised', 0.260725200176239),
 ('Remicade', 0.2591109275817871),
 ('nullification', 0.25816383957862854),
 ('Jaylah', 0.25565558671951294),
 ('Goenkaji', 0.25030404329299927),
 ('councelling', 0.24795880913734436),
 ('algebra', 0.2456909567117691),
 ('steps', 0.2454134076833725),
 ('deciding', 0.2434847205877304),
 ('Wan', 0.24254292249679565),
 ('mainsAs', 0.24145886301994324),
 ('sitting', 0.2404625564813614)]

In [11]:
word_vectors.most_similar('usa', topn=20)

[('arnd', 0.45440879464149475),
 ('rnkd', 0.4270583689212799),
 ('europe', 0.3883877098560333),
 ('Shud', 0.35802578926086426),
 ('canada', 0.3521583080291748),
 ('maidsin', 0.34342166781425476),
 ('melbourne', 0.3340461552143097),
 ('vit', 0.33228057622909546),
 ('nz', 0.32583385705947876),
 ('MedSchool', 0.3206053376197815),
 ('switzerland', 0.3175039291381836),
 ('rohini', 0.3133656978607178),
 ('Sinai', 0.3110540509223938),
 ('Canada', 0.30992138385772705),
 ('J2', 0.3089248538017273),
 ('saudi', 0.30678361654281616),
 ('ca', 0.3023535907268524),
 ('london', 0.30002766847610474),
 ('spain', 0.29272031784057617),
 ('nasa', 0.2923283576965332)]

In [53]:
word_vectors.similarity('mail', 'email')

0.31756443408502433

In [93]:
word_vectors.most_similar('max')

[('5volts', 0.4091402292251587),
 ('21volts', 0.40539640188217163),
 ('255', 0.40468814969062805),
 ('254', 0.38108837604522705),
 ('th8', 0.37641918659210205),
 ('KMs', 0.36274099349975586),
 ('Determine', 0.32687830924987793),
 ('th10', 0.3185482621192932),
 ('7x', 0.3072971999645233),
 ('horizontal', 0.3067953884601593)]

In [88]:
word_vectors.most_similar('Kohinoor')

[('Kingsley', 0.4966827630996704),
 ('dhampir', 0.48333466053009033),
 ('Koh', 0.4614385962486267),
 ('Meghnath', 0.444887638092041),
 ('victorian', 0.4388747215270996),
 ('Sant', 0.434789776802063),
 ('Noor', 0.4341884255409241),
 ('APJ', 0.4285680651664734),
 ('Babar', 0.4280354976654053),
 ('Gus', 0.42005109786987305)]

In [52]:
word_vectors.most_similar('mail')

[('USPS', 0.4655623137950897),
 ('mailbox', 0.4029355049133301),
 ('fax', 0.37872570753097534),
 ('FedEx', 0.3612022399902344),
 ('mailman', 0.3605952858924866),
 ('delivers', 0.3511347472667694),
 ('Parcel', 0.34227150678634644),
 ('transcript', 0.3388373553752899),
 ('Flipkart', 0.3243950605392456),
 ('mailed', 0.32292044162750244)]

In [12]:
word_vectors.most_similar('India')

[('Mumbai', 0.3737698793411255),
 ('subcontinent', 0.37103790044784546),
 ('Nepal', 0.3696967661380768),
 ('Pakistan', 0.3648681044578552),
 ('Indian', 0.3556423783302307),
 ('time', 0.34662944078445435),
 ('Bangalore', 0.33646100759506226),
 ('Kerala', 0.32918962836265564),
 ('in', 0.31129133701324463),
 ('Goa', 0.3068421483039856)]

In [29]:
word_vectors.most_similar('India')

[('c', 0.3641197085380554),
 ('Mumbai', 0.33095571398735046),
 ('Psychology', 0.3292388916015625),
 ('Australia', 0.32473915815353394),
 ('Infosys', 0.320180207490921),
 ('Chandigarh', 0.3180716633796692),
 ('IPC', 0.31621208786964417),
 ('Greece', 0.30715009570121765),
 ('Nepal', 0.30185842514038086),
 ('Kerala', 0.2966611087322235)]

In [3]:
# input_file     training file path (required)
# output         output file path (required)
# lr             learning rate [0.05]
# lr_update_rate change the rate of updates for the learning rate [100]
# dim            size of word vectors [100]
# ws             size of the context window [5]
# epoch          number of epochs [5]
# min_count      minimal number of word occurences [5]
# neg            number of negatives sampled [5]
# word_ngrams    max length of word ngram [1]
# loss           loss function {ns, hs, softmax} [ns]
# bucket         number of buckets [2000000]
# minn           min length of char ngram [3]
# maxn           max length of char ngram [6]
# thread         number of threads [12]
# t              sampling threshold [0.0001]
# silent         disable the log output from the C++ extension [1]
# encoding       specify input_file encoding [utf-8]

# Includes test data in unique_questions
# pd.Series(unique_questions[:100000]).to_csv('questions_data_1000.csv', index=False)
# NUM_PROC = 7
# model = fasttext.skipgram('questions_data.csv', 'model_full_data', dim=300, epoch=30, thread=NUM_PROC, word_ngrams=2)
# print model.words # list of words in dictionary

In [4]:
log_max_mem_usage()
model = fasttext.load_model('model_full_data.bin')
log_max_mem_usage()

Current all-time max memory: 136 MB
Current all-time max memory: 3345 MB


In [5]:
cosine_similarity(model["she is so India"], model["she's so India"])

array([[ 0.78911119]])

In [6]:
s1 = "she's so beautiful"
s2 = "she is pretty"
cosine_similarity(
    np.sum([model[i] for i in s1.split()], axis=0),
    np.sum([model[i] for i in s2.split()], axis=0),
#     model["she's so beautiful"]
)

array([[ 0.56863613]])

[-0.04152137041091919,
 0.13310760259628296,
 -0.10164530575275421,
 -0.30697593092918396,
 -0.17883634567260742,
 -0.20952075719833374,
 -0.0179507564753294,
 -0.125959575176239,
 0.14448928833007812,
 0.08694171905517578,
 -0.10691135376691818,
 0.007618176285177469,
 -0.29107901453971863,
 0.09584323316812515,
 -0.12915417551994324,
 0.06449639052152634,
 -0.020849671214818954,
 0.32004767656326294,
 0.26843661069869995,
 0.025120973587036133,
 0.1991555243730545,
 0.056532133370637894,
 0.03374658524990082,
 -0.11863112449645996,
 0.4082769453525543,
 -0.010660517029464245,
 0.3905940651893616,
 -0.3011779487133026,
 0.13229729235172272,
 -0.041249096393585205,
 0.03876839950680733,
 -0.04387875646352768,
 -0.04876932501792908,
 0.0779765248298645,
 -0.3986589312553406,
 0.03987919166684151,
 -0.11276043206453323,
 -0.05412406101822853,
 0.13856494426727295,
 -0.03304140642285347,
 0.0007616299553774297,
 0.164107084274292,
 -0.2787626087665558,
 -0.1146128699183464,
 -0.0439718849

In [4]:
# Consumes 6G when loaded
wvmodel = Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [28]:
(len(wvmodel.vocab) * 300 * 4.) / (1024 * 1000 * 1000)

3.515625

In [1]:
wvmodel

NameError: name 'wvmodel' is not defined